In [ ]:
# Импортируем необходимые объекты
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import json


# Метод отправки API запроса прямо в плагин
# Например для инициализации API ключа сервиса anti-captcha.com, необходимый для работы плагина
# Работает только на действующей HTML страничке,
# в нашем случае на https://antcpt.com/blank.html
# на страницах вроде about:blank запрос не пройдет
def acp_api_send_request(driver, message_type, data={}):
    message = {
        # всегда указывается именно этот получатель API сообщения
        'receiver': 'antiCaptchaPlugin',
        # тип запроса, например setOptions
        'type': message_type,
        # мерджим с дополнительными данными
        **data
    }
    # выполняем JS код на странице
    # а именно отправляем сообщение стандартным методом window.postMessage
    return driver.execute_script("""
    return window.postMessage({});
    """.format(json.dumps(message)))


# Инциируем объект опций для Хрома, чтобы иметь возможность подключить расширение
options = webdriver.ChromeOptions()
# Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали ранее
options.add_extension('C:/Users/user/for_selenium/anticaptcha-plugin_v0.3410.zip')

# Запускаем Браузер (Веб Драйвер Хрома) с указанием места скачивания самого файла драйвера
browser = webdriver.Chrome('C:/Users/user/for_selenium/chromedriver.exe', options=options)

# Переходим на пустую страницу для выполнения API запроса к плагину
browser.get('https://ras.arbitr.ru')

#здесь делаем закрытие объявления (в случае, если оно появилось)
#если объявление на сайте не появилось, то Selenium просто не найдет объект, отвечающий за него,
#и при попытке закрыть объявление сгенерируется исключение
#чтобы из-за этого исключения не падал весь дальнейший код, делаем обработку исключения с возможностью
#выполнения остатка кода
try:
    button_close = browser.find_element_by_class_name('b-promo_notification-popup-close')
    button_close.click()
except NoSuchElementException:
    pass

# Устанавливаем API ключ anti-captcha.com
# замените YOUR-ANTI-CAPTCHA-API-KEY на Ваш шестнадцатиричный ключ, который можно взять тут:
# https://anti-captcha.com/clients/settings/apisetup
acp_api_send_request(
    browser,
    'setOptions',
    {'options': {'antiCaptchaApiKey': 'da54f0f57f07a846c1682eb6e68e04d4'}}
)

# Три секунды паузы чтобы плагин проверил ключ на стороне anti-captcha.com
time.sleep(3)

# Переходим на тестовую форму с Рекапчей 2
#browser.get('https://antcpt.com/rus/information/demo-form/recaptcha-2.html')

# Тестовый ввод в инпут
browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]').send_keys('мошенничество')

# Самая важная чаcть: ждем не более 120 секунд пока индикатор антикаптчи с классом antigate_solver
# не получит класс solved, что означает что рекапча решена
WebDriverWait(browser, 120)
#.until(lambda x: x.find_element_by_css_selector('.antigate_solver.solved'))

# Отправляем форму
#earch_button = browser.find_element_by_xpath('//button[@alt="Найти"]')
browser.find_element_by_xpath('//button[@alt="Найти"]').click()